In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()

In [3]:
X = digits.data
y = digits.target

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 666)

In [5]:
from sklearn.neighbors import KNeighborsClassifier
sk_knn_clf = KNeighborsClassifier(n_neighbors=4,weights='uniform')
sk_knn_clf.fit(X_train, y_train)
sk_knn_clf.score(X_test, y_test)

0.9916666666666667

### Grid Search

In [6]:
#该参数是一个列表，列表中的元素是多个字典，也就是字典列表。
#一组不同的参数可以放到一个字典中。根据有没有P这个参数，又可以分出两个不同的字典
#一个字典对应一组网格搜索
#一组网格搜索里需要列出要遍历的参数对应的取值范围
#字典内的参数是乘法关系，字典间是加法关系

param_grid = [
    {
        'weights':['uniform'],  #键是参数的名称，值是一个列表，存储键对应的参数的所有可能的取值范围。
        'n_neighbors':[i for i in range(1,11)]
    },
    {
        'weights':['distance'],
        'n_neighbors':[i for i in range(1, 11)],
        'p':[i for i in range(1, 6)]  #参数p对weights = distance存在依赖，通过放在同一个字典中实现了这个依赖
    }
]

In [7]:
knn_clf = KNeighborsClassifier()

In [8]:
from sklearn.model_selection import GridSearchCV
#GridSearchCV()创建很多分类器，然后比较谁更好。创建的过程可以是并行的。
grid_search = GridSearchCV(knn_clf, param_grid)

In [9]:
%%time
grid_search.fit(X_train, y_train)

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


CPU times: user 59.1 s, sys: 379 ms, total: 59.5 s
Wall time: 59.8 s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [10]:
grid_search.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=3,
                     weights='distance')

In [11]:
grid_search.best_score_
#这里的评判标准和之前的不同

0.9853862212943633

In [12]:
grid_search.best_params_

{'n_neighbors': 3, 'p': 3, 'weights': 'distance'}

In [13]:
#拿到最佳参数对应的分类器，best_estimator_返回一个KNN对象
knn_clf = grid_search.best_estimator_

In [14]:
knn_clf.score(X_test, y_test)

0.9833333333333333

#GridSearchCV()创建很多分类器，然后比较谁更好。创建的过程可以是并行的。n_jobs表示使用的CPU的个数，-1表示不考虑有几个，全都用。
#因为搜索的时间会很长，可以及时了解搜索状态和搜索信息是非常重要的。verbose表示每次搜索的输出信息量。数值越大，信息越详细。

In [16]:
%%time
grid_search = GridSearchCV(knn_clf, param_grid, n_jobs= -1, verbose=4 )
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


/Users/apple/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   11.1s


CPU times: user 602 ms, sys: 122 ms, total: 724 ms
Wall time: 19.8 s


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   19.7s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=3, p=3,
                                            weights='distance'),
             iid='warn', n_jobs=-1,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=4)